# Carga de Datos

Hacemos distincción para posible estudio de rendimientos en la forma de guardado en disco y su lectura.

#### Carga Desde Binarios

In [ ]:
\ts historical: get `:Data/DataWarehouse/Tables/historical

In [ ]:
\ts resume_etf: get `:Data/DataWarehouse/Tables/resume_etf

#### Carga Desde Tablas Splayed

In [ ]:
\ts load `:Data/DataWarehouse/Tables/historicalSplayed/sym;
\ts historicalS: get `:Data/DataWarehouse/Tables/historicalSplayed/historical

In [ ]:
\ts load `:Data/DataWarehouse/Tables/resume_etfSplayed/sym;
\ts resume_etfS: get `:Data/DataWarehouse/Tables/resume_etfSplayed/resume_etf

# Cálculos

In [ ]:
meta historical

In [ ]:
select from historical where m200sma <> 0N, ticker=`EWJ, date>2020.01.01

In [ ]:
select days:count i by year: `year$date, ticker from historical

# Visualización de Datos

Para este apartado usaremos las sinergias entre KDB+/Q y Python, que al usar Jupyter Notebook se nos facilita aún más esa integración.

Usaremos el módulo y biblioteca por excelencia para esta representación gráfica en Python, [Matplotlib](https://matplotlib.org/stable/).

En primer lugar cargaremos en las variables **plt** y **mdates** los módulos de Python más usados para la representación gráfica y manipulación de fechas.

In [ ]:
\l p.q
plt:.p.import[`matplotlib;`:pyplot]
mdates:.p.import[`matplotlib;`:dates]

### Generación de Gráficos Históricos y de Variación de Volumen

In [ ]:
\l QFunctions/graphics.q
\l QFunctions/queries.q
etfs: exec ticker from resume_etf
/
{in_outflow_g_F[x;2020.01.01]} each etfs;
{historical_g_F[x;2020.01.01]} each etfs;
{in_outflow_g[x]} each etfs;
{historical_g[x]} each etfs;
\

In [ ]:
{in_outflow_g_F[x;2020.01.01]} each etfs;

In [ ]:
{historical_g_F[x;2020.01.01]} each etfs;

In [ ]:
{in_outflow_g[x]} each etfs;

In [ ]:
{historical_g[x]} each etfs;

In [ ]:
// FUNCION PARA PROBAR ELEMENTOS DE MATPLOTLIB
/
historical_g_F_1:{[ETF;DATE]
    plt[`:figure][(`figsize);(22;14)]; 

    plt[`:plot][ax_x_F[ETF;DATE];ax_y_F[ETF;DATE];"y-"];
    plt[`:plot][aSMA_x_F[ETF;DATE];aSMA_y_F[ETF;DATE];"r-"];
    plt[`:plot][aMSMA_x_F[ETF;DATE];aMSMA_y_F[ETF;DATE];"g-"];

    ax: plt[`:gca][];
    
    ax[`:xaxis][`:set_major_locator][mdates[`:AutoDateLocator][]];

    plt[`:xticks][;][;][(`rotation;45)];
    plt[`:gcf][][`:autofmt_xdate][];  

    name: string ETF;
    title: "HISTORICAL - ",name," FROM ",(string DATE);
    plt[`:title][title];
    plt[`:xlabel]["DATE"];
    plt[`:ylabel]["VALUE UP/DOWN"]; 
    plt[`:legend][("NAV VALUE";"Daily SMA 200";"Monthly SMA 200")];
    plt[`:show][];
    plt[`:clf][];
 };
historical_g_F_1[`EWA;2020.01.01]
\

# Modelo de Estrategia de Inversión en ETFs

En este apartado realizaremos el soporte para la elección de ETF.

En el anterior Notebook hemos realizado todos los cálculos necesarios para ahora poder selecionar y evaluar los ETF para nuestra cartera.

La técnica de selección, ponderación, etf es motivado por mi TFG de Administración de Empresas y los motivos de elección de cada apartado está explicado en él.


## Benchmark por Categoría

Se realizará el mismo benchmark para todos los ETF de la misma manera. Para poder conseguir el *Riesgo Diversificado* el mejor resultado por categoría:
> - **Mercado** en el que opera.
> - **Región** en la que opera.
> - **Clase de Equity** del que está compuesto el ETF.

## Benchmark para Modelo de Inversión

Se dará una valoración a cada elemento de:
- Tendencia del Mercado respecto a su SMA: **?%**
- Tendencia de Valoración respecto a su mSMA: **?%**
- Volatilidad a 1 año: **?%**
- Volatilidad a 6 meses: **?%**
- Volatilidad a 3 meses: **?%**
- Correlación Inflow/Outflow con Nav_Value: **?%**
- Distancia Temporal de su máximo + % valor actual respecto a él: **?%**
- Distancia Temporal de su mínimo + % valor actual respecto a él: **?%**
- Rendimiento los 5 años: **?%**
- Rendimiento los 1 año: **?%**
- Rendimiento los 6 meses: **?%**
- Drawdown a 5 años: **?%**
- Drawdown a 1 año: **?%**
- Drawdown a 6 mesess: **?%**

In [ ]:
correlations_: {
    t: exec ticker from resume_etf;
    p: exec nav_value by ticker from historical where in_out_flow_per <> 0N;
    io: exec in_out_flow_per by ticker from historical where in_out_flow_per <> 0N;
    c: {p[x] cor io[x]} each t;
    flip `ticker`correlation!((tickers; c))

 }
correlations:{ [DATE]
    t: exec ticker from resume_etf;
    p: exec nav_value by ticker from historical where in_out_flow_per <> 0N, date > DATE;
    io: exec in_out_flow_per by ticker from historical where in_out_flow_per <> 0N, date > DATE;
    c: {p[x] cor io[x]} each t;
    flip `ticker`correlation!((tickers; c))
 }

In [ ]:
table:()
table: update ticker: ticker from select ticker from resume_etf
table: update correlation: correlations_[][`correlation] from table

# Extras + info futuro


In [ ]:
count select ticker from resume_etf where market = `$"Developed"
select from historical where ticker = `EWA, m200sma <> 0N
count select ticker from resume_etf where market = `Emerging

//Filtrado  mezclando las dos tablas
select from historical where ticker in exec ticker from resume_etf where market = `$"Emerging"
//Lo mismo
show a: select ticker from resume_etf where market = `$"Developed"
select from historical where ticker in a[`ticker]

select distinct equity_class from resume_etf
select distinct region from resume_etf

//maximo aumento
select from historical where daily_retns=(max; daily_retns) fby ticker